In [1]:
import numpy as np
import matplotlib.image as img
import matplotlib.pyplot as plt
import cv2

In [ ]:
###cap = cv2.VideoCapture(2)


while(1):

    # Take each frame
    ###_, frame = cap.read()
    
    frame = cv2.imread("sample_arena_table_2.PNG")

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    h = 210
    s = 0.52
    v = 0.87
    #blue blocks h = 210°, s = 0.52, v = 0.87
    #red line h = 1° s = 0.44  v = 0.93
    
    # define range of green color in HSV
    lower_green = np.array([(h-20)*0.5,255*max(s-0.2,0),255*max(v-0.2,0)])
    upper_green = np.array([(h+20)*0.5,255*min(s+0.2,1),255*min(v+0.2,1)])

    # Threshold the HSV image to get only blue colours
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)

    cv2.imshow('frame',frame)
    cv2.imshow('mask',mask)
    cv2.imshow('res',res)
    k = cv2.waitKey(5) & 0xFF
    if k == 5:
        break

cv2.destroyAllWindows()

In [214]:
cv2.destroyAllWindows()